# Compare sequential vs. squeezed implementations of `multiply_dimensionwise`, 2D

At first, you need to run the benchmarks by executing
```
julia -e benchmark_multiply_dimensionwise.jl
```
Then, you can generate the plots by running this notebook.

In [ ]:
using Printf

using BSON
using PyCall
colors = pyimport("matplotlib.colors")
import PyPlot; plt = PyPlot


function plot_benchmarks_helper(n_vars_list, n_nodes_in_list, title, values, colorbar_label)
  x = ones(length(n_nodes_in_list)) .* n_vars_list'
  y = n_nodes_in_list .* ones(length(n_vars_list))'

  exponent = max(
    maximum(abs, round.(Int, log10.(extrema(values)), RoundUp)),
    maximum(abs, round.(Int, log10.(extrema(values)), RoundDown)))
  fig, ax = plt.subplots(1, 1)
  plt.imshow(transpose(values),
      origin="lower", extent=(n_vars_list[1]-0.5, n_vars_list[end]+0.5,
          n_nodes_in_list[1]-0.5, n_nodes_in_list[end]+0.5),
      cmap="seismic", norm=colors.LogNorm(vmin=10.0^(-exponent), vmax=10.0^(+exponent)),
  #     norm=colors.LogNorm()
  )
  plt.locator_params(axis="x", nbins=length(n_vars_list))
  plt.locator_params(axis="y", nbins=length(n_nodes_in_list))
  plt.colorbar(label=colorbar_label)
  plt.axis("Image")
  plt.xlabel("n_variables_total")
  plt.ylabel("n_nodes_in")
  plt.title(title * @sprintf("\nextrema: (%.1e, %.1e)", extrema(values)...))
  fig
end

function plot_benchmarks(datafile, title=datafile)
  BSON.@load datafile n_vars_list n_nodes_in_list sequential_dynamic sequential_static sequential_nexpr sequential_dynamic_prealloc sequential_static_prealloc squeezed_dynamic squeezed_static
#   @assert minimum(sequential_static ./ sequential_dynamic) >= 1
#   @assert minimum(sequential_static_prealloc ./ sequential_dynamic_prealloc) >= 1
  @assert minimum(sequential_dynamic_prealloc ./ sequential_dynamic_prealloc) <= 1

  values = sequential_static ./ sequential_dynamic; colorbar_label = "sequential_static ./ sequential_dynamic"
  fig = plot_benchmarks_helper(n_vars_list, n_nodes_in_list, title, values, colorbar_label)
  values = squeezed_static ./ squeezed_dynamic; colorbar_label = "squeezed_static ./ squeezed_dynamic"
  fig = plot_benchmarks_helper(n_vars_list, n_nodes_in_list, title, values, colorbar_label)

  values = squeezed_dynamic ./ sequential_dynamic; colorbar_label = "squeezed_dynamic ./ sequential_dynamic"
  fig = plot_benchmarks_helper(n_vars_list, n_nodes_in_list, title, values, colorbar_label)
  values = squeezed_static ./ sequential_dynamic; colorbar_label = "squeezed_static ./ sequential_dynamic"
  fig = plot_benchmarks_helper(n_vars_list, n_nodes_in_list, title, values, colorbar_label)

  values = sequential_dynamic_prealloc ./ sequential_dynamic; colorbar_label = "sequential_dynamic_prealloc ./ sequential_dynamic"
  fig = plot_benchmarks_helper(n_vars_list, n_nodes_in_list, title, values, colorbar_label)

  values = squeezed_dynamic ./ sequential_dynamic_prealloc; colorbar_label = "squeezed_dynamic ./ sequential_dynamic_prealloc"
  fig = plot_benchmarks_helper(n_vars_list, n_nodes_in_list, title, values, colorbar_label)
  values = squeezed_static ./ sequential_dynamic_prealloc; colorbar_label = "squeezed_static ./ sequential_dynamic_prealloc"
  fig = plot_benchmarks_helper(n_vars_list, n_nodes_in_list, title, values, colorbar_label)

  return nothing
end

In [ ]:
arch = "i7-8700K_"
# arch = "i7-6850K_"
# arch = "XeonGold6230_"
plot_benchmarks(arch * "2D_nVarTotal_nNodesIn.bson", "n_nodes_out = n_nodes_in")

In [ ]:
plot_benchmarks(arch * "2D_nVarTotal_2nNodesIn.bson", "n_nodes_out = 2*n_nodes_in")

In [ ]:
plot_benchmarks(arch * "3D_nVarTotal_nNodesIn.bson", "n_nodes_out = n_nodes_in")

In [ ]:
plot_benchmarks(arch * "3D_nVarTotal_2nNodesIn.bson", "n_nodes_out = 2*n_nodes_in")

# WIP: Development. DO NOT RUN THE NEXT CELL